<a href="https://colab.research.google.com/github/DungDuc/MEDNet/blob/main/MEDNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Readme
In this notebook, you could:

* Access the data and models from the following link:

* Copy data and checkpoints files to your system, e.g. Google Drive.

* Use the data and models for testing or re-experiment, e.g. mount the Google Drive and run the following cells on Google Colab.


# Import

In [ ]:
# import library
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, ZeroPadding2D, Concatenate, Multiply, Flatten, MaxPooling2D
from tensorflow.keras.layers import Conv1D, Conv2D, Conv2DTranspose, MaxPooling2D, AveragePooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D, Attention
from tensorflow.keras.layers import BatchNormalization, Dropout, Activation, ReLU, Dense
from tensorflow.keras.layers import Add, Multiply, Reshape, DepthwiseConv2D, UpSampling2D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import Sequence
from tensorflow.keras.metrics import *
import tensorflow.keras.backend as K

from google.colab.patches import cv2_imshow
from IPython.display import clear_output

import glob
import os
import cv2
import time
import random

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import yaml
import xml.etree.ElementTree as ET
import struct
import csv



# API

In [ ]:
# class labels and colors
AllTags = ['Reg1', 'Reg2', 'Reg3',\
        'Reg4', 'Reg5', 'Reg6',\
        'Reg7', 'Reg8', 'Reg9']

#Colors = [[255,0,0], [0,255,0], [0,0,255],\
#Colors = [[0,255,0], [0,0,255], [255,0,0],\
Colors = [[0,0,255], [255,0,0], [0,255,0], \
          [255,255,0], [255,0,255], [0,255,255],\
          [128,0,0], [0,128,0], [0,0,128],\
          [255,255,255]]
MaxColor = len(Colors)-1

def set_label(Tags):
    # class name and colors
    nColor = len(Colors)
    for i in range(len(Tags)):
        class_names.append(Tags[i])
        label_values.append(Colors[i%nColor])
    # background
    class_names.append('Background')
    label_values.append([255,255,255])

class_names, label_values = [],[]
set_label(AllTags)

# one-hot and colour coding
def one_hot_it(code, num_classes):
    masks = []
    for i in range(num_classes):
        mi = np.uint8((code==i))*1
        masks.append(mi)
    return np.stack(masks, axis=-1)

def reverse_one_hot(image):
    x = np.argmax(image, axis = -1).astype(np.uint16)
    return x

def colour_code_segmentation(img, MaxClass=0):
    image = img.copy()
    colour_codes = np.array(label_values).astype(np.uint8)
    if MaxClass > 0:
      image[image==MaxClass] = MaxColor
    x = colour_codes[image.astype(np.uint16)]
    return x


class Region(object):
    def __init__(self, tag = None, xmin=None, ymin=None, xmax=None, ymax=None, poly=None):
        self.tag = tag
        self.xmin = xmin
        self.ymin = ymin
        self.xmax = xmax
        self.ymax = ymax
        self.poly = poly
    def iou(self, other):
        if self.poly is not None and other.poly is not None:
            return poly_iou(self.poly, other.poly)
        else:
            return 0

def bb_iou(regA, regB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xLeft = max(regA.xmin, regB.xmin)
    yLeft = max(regA.ymin, regB.ymin)
    xRight = min(regA.xmax, regB.xmax)
    yRight = min(regA.ymax, regB.ymax)

    if xLeft > xRight or yLeft > yRight:
      return 0

    # compute the area of intersection rectangle
    interArea = max(0, xRight - xLeft + 1) * max(0, yRight - yLeft + 1)
    # compute the area of both the prediction and ground-truthrectangles
    boxAArea = (regA.xmax - regA.xmin + 1) * (regA.ymax - regA.ymin + 1)
    boxBArea = (regB.xmax - regB.xmin + 1) * (regB.ymax - regB.ymin + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou


def GetFiles(Folders, ext):
    Files = []
    for folder in Folders:
        files = glob.glob(folder + '/*.' + ext)
        files.sort()
        Files.extend(files)
    return Files

def ResizeImg(img, height, width):
    img_h, img_w = img.shape[0], img.shape[1]
    scale = min(float(height) / img_h, float(width) / img_w)

    img = cv2.resize(img, (int(scale*img_w), (int(scale*img_h))))
    if img.shape[1] < width:
        img = cv2.copyMakeBorder(img, 0, 0, 0, width - img.shape[1], cv2.BORDER_REPLICATE)
    if img.shape[0] < height:
        img = cv2.copyMakeBorder(img, 0, height - img.shape[0], 0, 0, cv2.BORDER_REPLICATE)

    if img.ndim < 3:
        img = np.expand_dims(img, axis=-1)

    return img, scale

def Binary(ImgIn):
    if ImgIn.shape[-1] == 3:
      gray = cv2.cvtColor(ImgIn, cv2.COLOR_RGB2GRAY)
    else:
      gray = ImgIn[...,0]

    #ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 9)

    return thresh

def ViewImgs(Imgs):
    fig = plt.figure(figsize=(15, 15))
    for i in range(len(Imgs)):
        plt.subplot(1, len(Imgs), i+1)
        if Imgs[i].ndim == 3:
            if  Imgs[i].shape[-1] == 3:
              plt.imshow(Imgs[i])
            else:
              plt.imshow(Imgs[i][...,0], cmap='gray', vmin=0, vmax=255)
        else:
            plt.imshow(Imgs[i], cmap='gray', vmin=0, vmax=255)
    plt.show()

def SaveErrs(Errs, FileName, ErrImgs_H=2048, ErrImgs_W=1024):
    print('SaveErrs: ', len(Errs))
    ErrImgs = np.zeros((ErrImgs_H, ErrImgs_W, 3), np.uint8)
    ErrX, ErrY = 0, 0
    ErrH = 0
    m = 3
    for err in Errs:
        rgH, rgW = err.shape[0], err.shape[1]

        if ErrX + rgW > ErrImgs_W:
            # next line
            ErrX = 0
            ErrY += ErrH + m
            ErrH = 0

        if ErrY + rgH < ErrImgs_H and ErrX + rgW < ErrImgs_W:
            ErrImgs[ErrY:ErrY+rgH, ErrX:ErrX+rgW,...] = err
            ErrX += rgW + m
            if ErrH < rgH:
                ErrH = rgH

    SaveImgs([ErrImgs], FileName)


def SaveImgs(Imgs, FileName):
    for i, img in enumerate(Imgs):
      cv2.imwrite(FileName+str(i)+'.png', img)


def MasksImg(masks, img=None):
    masks_img = colour_code_segmentation(reverse_one_hot(masks), MaxClass=masks.shape[-1]-1)
    # resize mask
    if img is not None:
        if img.shape[0] != masks_img.shape[0] or img.shape[1] != masks_img.shape[1]:
            #masks_img,_ = ResizeImg(masks_img, img.shape[0], img.shape[1])
            img,_ = ResizeImg(img, masks_img.shape[0], masks_img.shape[1])
        masks_img = ((0.5 * img) + (0.5 * masks_img)).astype("uint8")
        masks_img = np.minimum(masks_img, img)

    return masks_img


def hex_to_double(hStr):
    return struct.unpack("d", struct.pack("Q",int("0x"+hStr, 16)))[0]

# adjust ground truth
def AdjustGTs(regions, bw):
  for i, rg in enumerate(regions):
      regions[i] = AdjustGT(rg, bw)
  return regions


def AdjustGT(rg, bw):
    # left-right
    m = 4
    rg.xmin -= m
    rg.xmax += m
    roi = bw[rg.ymin:rg.ymax, rg.xmin:rg.xmax]
    hist = np.sum((255-roi)/255,axis=0)
    len_h = len(hist)

    # find the closest white line
    left_ws = np.array([i for i in range(0,2*m) if (hist[i] == 0 and i < len_h//2)])
    right_ws = np.array([i for i in range(len_h-2*m, len_h) if (hist[i] == 0 and i > len_h//2)])

    left, right = m, len_h-m
    if len(left_ws) > 0:
        left = min(left_ws, key=lambda x:abs(x-left))
    if len(right_ws) > 0:
        right = min(right_ws, key=lambda x:abs(x-right))


    while left < right and hist[left] == 0:
        left += 1
    left -= 1
    while right > left and hist[right] == 0:
        right -= 1
    right += 1

    rg.xmin += left
    rg.xmax -= (len_h - right)

    # top-bottom
    m = 8
    rg.ymin -= m
    rg.ymax += m
    roi = bw[rg.ymin:rg.ymax, rg.xmin:rg.xmax]
    hist = np.sum((255-roi)/255,axis=1)
    len_h = len(hist)

    # find the closest white line
    top_ws = np.array([i for i in range(0,2*m) if (hist[i] == 0 and i < len_h//2)])
    bot_ws = np.array([i for i in range(len_h-2*m, len_h) if (hist[i] == 0 and i > len_h//2)])

    top, bot = m, len_h-m
    if len(top_ws) > 0:
        top = min(top_ws, key=lambda x:abs(x-top))
    if len(bot_ws) > 0:
        bot = min(bot_ws, key=lambda x:abs(x-bot))

    while top < bot and hist[top] == 0:
        top += 1
    top -= 1
    while bot > top and hist[bot] == 0:
        bot -= 1
    bot +=1

    rg.ymin += top
    rg.ymax -= (len_h - bot)

    return rg

# data normalization
Norm = lambda x: np.float32(x)/np.max(x)

class InstanceNormalization(tf.keras.layers.Layer):
    """Instance Normalization Layer (https://arxiv.org/abs/1607.08022)."""
    def __init__(self, epsilon=1e-5):
        super(InstanceNormalization, self).__init__()
        self.epsilon = epsilon

    def build(self, input_shape):
        self.scale = self.add_weight(name='scale', shape=input_shape[-1:],
                                     initializer=tf.random_normal_initializer(1., 0.02),
                                     trainable=True)

        self.offset = self.add_weight(name='offset', shape=input_shape[-1:], initializer='zeros',
                                      trainable=True)

    def call(self, x):
        mean, variance = tf.nn.moments(x, axes=[1, 2], keepdims=True)
        inv = tf.math.rsqrt(variance + self.epsilon)
        normalized = (x - mean) * inv
        return self.scale * normalized + self.offset


def UpScaling(x, scale):
    x = UpSampling2D(size=[scale, scale], interpolation='bilinear')(x)
    return x


# Data label

In [ ]:
# single-label masks
def Reg2MaskSL(Regions, img):
    img_h, img_w = img.shape[0:2]
    seg_bg = (np.zeros((img_h, img_w), np.uint8) * 255)
    seg_masks = []
    for c in range(num_classes):
        # class mask
        mask_c = np.zeros((img_h, img_w), np.uint8)
        for reg in Regions:
            if int(reg.tag) == c:
                mask_c[reg.ymin:reg.ymax, reg.xmin:reg.xmax] = 255
        # augmented mask
        seg_masks.append(mask_c)
        seg_bg = np.maximum(seg_bg, mask_c)

    # all classes
    seg_masks.append(255 - seg_bg)

    return np.stack(seg_masks, axis=-1)


# multiple-label masks
def Reg2MaskML(Regions, bw):
    img_h, img_w = bw.shape[0:2]
    seg_fg = (np.zeros((img_h, img_w), np.uint8) * 255)
    det_fg = (np.zeros((img_h, img_w), np.uint8) * 255)

    seg_masks, detect_masks = [], []
    for c in range(num_classes):
        # class mask
        mask_c = np.zeros((img_h, img_w), np.uint8)
        for reg in Regions:
            if int(reg.tag) == c:
                mask_c[reg.ymin:reg.ymax, reg.xmin:reg.xmax] = 255

        # augmented mask
        fg_c = mask_c
        margin = 4
        if c == 1 or c == 0:
            fg_c = np.uint8(((255-bw)*(mask_c==255)))
            fg_c = cv2.dilate(fg_c, np.ones((2*margin+1, 2*margin+1), np.uint8))
            fg_c = cv2.morphologyEx(fg_c, cv2.MORPH_CLOSE, np.ones((2*margin+1, 4*margin+1), np.uint8))
            fg_c *= (mask_c==255)

        # detection mask
        det_fg = np.maximum(det_fg, fg_c)

        # segmentation mask
        fg_c = mask_c
        seg_masks.append(fg_c)
        seg_fg = np.maximum(seg_fg, fg_c)

    # all classes
    seg_masks.append(255 - seg_fg)
    detect_masks.append(det_fg)
    detect_masks.append(255 - det_fg)

    AllMask = [det_fg]
    AllMask.extend(seg_masks)

    #return np.stack(seg_masks, axis=-1), np.stack(detect_masks, axis=-1)
    return np.stack(AllMask, axis=-1), np.stack(detect_masks, axis=-1)


def ViewData(data_fn, LoadDataFunct):
    img, y_seg, y_detect, RegionsGT = MEDLoadData(data_fn, LoadDataFunct, INPUT_H, INPUT_W, INPUT_C)

    y_img = img
    if img.shape[-1] == 1:
        y_img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    for rg in RegionsGT:
        cv2.rectangle(y_img, (int(rg.xmin), int(rg.ymin)), (int(rg.xmax), int(rg.ymax)), Colors[int(rg.tag)], 2)

    y_sseg = Reg2MaskSL(RegionsGT, img)
    y_sseg_img = MasksImg(y_sseg, img)

    y_seg_img = MasksImg(y_seg, img)
    y_det_img = MasksImg(y_detect, img)

    cv2_imshow(y_img)
    #cv2_imshow(y_sseg_img)
    #cv2_imshow(y_det_img)
    #cv2_imshow(y_seg_img)
    ViewImgs([y_img, y_det_img, y_seg_img])


def MEDLoadData(data_fn, LoadDataFunct, img_h=0, img_w=0, img_c=0):
    # image and GT
    img, RegionsGT = LoadDataFunct(data_fn)
    if img is not None:
        # size normalize
        if img_h > 0 and img_w > 0:
            img, scale = ResizeImg(img, img_h, img_w)

            for i, r in enumerate(RegionsGT):
                r.xmin = int(r.xmin*scale)
                r.xmax = int(r.xmax*scale)
                r.ymin = int(r.ymin*scale)
                r.ymax = int(r.ymax*scale)
                RegionsGT[i] = r

        # adjust ground truth
        bw = Binary(img)
        RegionsGT = AdjustGTs(RegionsGT, bw)

        # regions to mask
        y_seg, y_detect = Reg2MaskML(RegionsGT, bw)

        if img_c == 1:
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            img = np.expand_dims(img, axis=-1)

        return img, y_seg, y_detect, RegionsGT

    return None, None, None, None

# MEDNet

In [ ]:
def MEDNet(input_shape, nClass, nFeatures=128,
           scale1=1, scale2=3, ext=3, first_filters=32, nConv=3):
    # input
    inputs = Input(shape=input_shape)

    # features
    features = FeatExtractor(inputs, depth=scale2+ext, first_filters=first_filters, nConv=nConv)
    s1_feats, s2_feats = [], []
    for i in range(ext):
        s1_feats.append(UpScaling(features[scale1+i], 2**(i+1)))
        s2_feats.append(UpScaling(features[scale2+i], 2**(i+1)))

    s1_feats = Concatenate()(s1_feats)
    s1_feats = ConvBlocks(s1_feats, nFeatures, kernel_size=1, stride=1, nConv=1)
    s2_feats = Concatenate()(s2_feats)
    s2_feats = ConvBlocks(s2_feats, nFeatures, kernel_size=1, stride=1, nConv=1)

    # scale1 detection map
    s1_feats, s1_map = SegHead(s1_feats, nFeatures, out_ch=2, nConv=0)
    s1_map = Activation('sigmoid', name="y1")(s1_map)

    # scale2 segmentation map
    s2_feats, s2_map = SegHead(s2_feats, nFeatures, out_ch=nClass+2, nConv=0)
    s2_map = Activation('sigmoid', name="y2")(s2_map)

    # combined feature maps
    s2_feats = UpScaling(s2_feats, 2 ** (scale2 - scale1))
    s12_feats = Concatenate()([s1_feats, s2_feats])
    s12_feats = ConvBlocks(s12_feats, nFeatures, kernel_size=1, stride=1, nConv=1)

    # final segmentation map
    s12_feats, seg_map = SegHead(s12_feats, nFeatures, out_ch=nClass+2, nConv=3)
    seg_map = UpScaling(seg_map, 2 ** scale1)
    seg_map = Activation('sigmoid', name="y")(seg_map)

    return Model(inputs=inputs, outputs=[seg_map, s1_map, s2_map])


def SegHead(x, filters, out_ch=1, nConv=1, DropRate=0.2):
    if nConv > 0:
        x = ConvBlocks(x, filters, kernel_size=(3,3), nConv=nConv, DropRate=DropRate)

    seg_map = Conv2D(out_ch, kernel_size=3, strides=1, padding='same')(x)
    return x, seg_map


def FeatExtractor(x, depth=5, first_filters=32, nConv=2):
    features = []
    filters = first_filters
    for i in range(depth):
        if i == 0:
            x = ConvBlocks(x, filters, kernel_size=(7,5), stride=2, nConv=nConv)
            filters *= 2
            nConv += 1
        else:
            x = K_ConvBlocks(x, filters, [(3, 3), (3, 7)], nConv=nConv)
            x = MaxPooling2D()(x)

        features.append(x)

    return features


# convolution blocks
def K_ConvBlocks(x, filters, kernel_sizes, nConv=2, NormType='instance', DropRate=0.2):
    maps = []
    for k in kernel_sizes:
        m = ConvBlocks(x, filters//2, kernel_size=k, stride=1, nConv=nConv, NormType=NormType, DropRate=DropRate)
        maps.append(m)
    map = Concatenate()(maps)
    map = ConvBlocks(map, filters, kernel_size=1, stride=1, nConv=1, NormType=NormType, DropRate=DropRate)

    return map

def ConvBlocks(x, filters, kernel_size=(3, 3), dilation_rate=1, stride=1, nConv=1, NormType='instance', DropRate=0.2):
    # first conv
    x = Conv_BN_Act(filters, kernel_size=kernel_size, dilation_rate=dilation_rate,stride=stride,
                    NormType=NormType, DropRate=DropRate)(x)
    # stride-1 convolution
    for i in range(1, nConv):
        x = Conv_BN_Act(filters, kernel_size=kernel_size, stride=1, NormType=NormType, DropRate=DropRate)(x)

    return x

# Basic convolution block
def Conv_BN_Act(filters, kernel_size=(3, 3), dilation_rate=(1, 1), stride=1, NormType='instance', DropRate=0.2):
    f = Sequential()

    f.add(Conv2D(filters, kernel_size, strides=stride, dilation_rate=dilation_rate, padding='same', use_bias=False))

    if NormType.lower() == 'batch':
        f.add(BatchNormalization())
    elif NormType.lower() == 'instance':
        f.add(InstanceNormalization())

    f.add(ReLU())

    if DropRate > 0.0:
        f.add(Dropout(DropRate))

    return f


In [ ]:
# defaut variables
INPUT_H, INPUT_W, INPUT_C = 1472, 1024, 1
input_shape = [INPUT_H, INPUT_W, INPUT_C]
num_classes = 2

scale1, scale2, ext = 1, 3, 3
nConv, first_filters = 3, 64
nFeatures = 128

#act_funct='softmax'
#loss_funct = 'categorical_crossentropy'
#metric = 'accuracy'

act_funct='sigmoid'
loss_funct = 'binary_crossentropy'
metric = 'binary_accuracy'

Losses = {
    "y": loss_funct,
    "y1": loss_funct,
    "y2": loss_funct,
}

def BuildModel():
    model = MEDNet(input_shape, num_classes, nFeatures=nFeatures,
                   scale1=scale1, scale2=scale2, ext=ext, first_filters=first_filters, nConv=nConv)

    #optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001)

    #radam = tfa.optimizers.RectifiedAdam()
    #ranger = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)
    #optimizer = ranger

    model.compile(loss=Losses, optimizer=optimizer, metrics=[metric])

    return model


# test
model = BuildModel()
model.summary()
#tf.keras.utils.plot_model(model, show_shapes=True)


# Test API

In [ ]:
MIN_W, MIN_H = 5, 7

def Predict(model, img):
    x = img
    if INPUT_C == 1 and len(x.shape) == 3 and x.shape[-1] == 3:
        x = cv2.cvtColor(x, cv2.COLOR_RGB2GRAY)
    if x.shape[0] != INPUT_H or x.shape[1] != INPUT_W:
        x, scale = ResizeImg(x, INPUT_H, INPUT_W)
    x = Norm(x)
    x = np.expand_dims(x, axis=0)
    return model.predict(x, verbose=0)


def SampleTest(model, img_fn, num_classes=1, FileName=None, thresh=0.7, debug=True):
    Imgs = []
    # load data
    #img, all_masks, RegionsGT = LoadData(img_fn, INPUT_H, INPUT_W, INPUT_C)
    img, seg_masks, det_masks, RegionsGT = MEDLoadData(img_fn, LoadDataFunct, INPUT_H, INPUT_W, INPUT_C)
    if img is None:
       return None, None, None, None, None

    y_img = MasksImg(seg_masks, img)
    #y_img = MasksImg(seg_masks, 255-img)
    #ViewImgs([y_img])

    # predict
    #yy_seg, yy_seg2 = Predict(model, img)
    yy_seg, yy_seg1, yy_seg2 = Predict(model, img)
    yy_seg = yy_seg[0]
    yy_det_masks = yy_seg1[0]
    yy_seg[yy_seg >= 0.5] = 1

    # detection
    #RegionsPD = GetRegionsPD(yy_seg, num_classes, img)
    RegionsPD = GetRegionsPD(yy_seg[...,1:], num_classes, img)
    #RegionsPD = GetRegionsPD(yy_seg[...,1:], num_classes, 255-img)

    # performance
    IoUs, TPRegs, nPos, nPred, nTP = MatchRegs(RegionsGT, RegionsPD, num_classes, thresh)

    # errors
    yy_seg[yy_seg >= 0.5] = 1
    #yy_img = MasksImg(yy_seg, 255-img)
    yy_img = MasksImg(yy_seg, img)

    for rg in RegionsGT:
        #cv2.rectangle(yy_img, (int(rg.xmin), int(rg.ymin)), (int(rg.xmax), int(rg.ymax)), [0,0,255], 1)
        cv2.rectangle(yy_img, (int(rg.xmin), int(rg.ymin)), (int(rg.xmax), int(rg.ymax)), Colors[int(rg.tag)+1], 1)
    for rg in RegionsPD:
        cv2.rectangle(yy_img, (int(rg.xmin), int(rg.ymin)), (int(rg.xmax), int(rg.ymax)), Colors[int(rg.tag)+1], 1)

    MissErrs, WrongErrs = [], []
    m = 8
    for (iou, rg) in zip(IoUs, RegionsGT):
        if iou < thresh:
            cv2.putText(yy_img, "{:.1f}".format(iou*100), (rg.xmin, rg.ymin-3), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,0,255), 1)
            # err image
            if int(rg.tag) == 0:
                errImg = yy_img[rg.ymin-m:rg.ymax+m, rg.xmin-m:rg.xmax+m,...]
                MissErrs.append(errImg)

    IoUs2, TPRegs2, nPos2, nPred2, nTP2 = MatchRegs(RegionsPD, RegionsGT, num_classes, thresh)
    for (iou, rg) in zip(IoUs2, RegionsPD):
        #cv2.rectangle(yy_img, (int(rg.xmin), int(rg.ymin)), (int(rg.xmax), int(rg.ymax)), Colors[int(rg.tag)+1], 1)
        if iou < thresh:
            cv2.putText(yy_img, "{:.1f}".format(iou*100), (rg.xmin, rg.ymin-3), cv2.FONT_HERSHEY_SIMPLEX, 0.3, Colors[int(rg.tag)], 1)
            if int(rg.tag) == 0:
                errImg = yy_img[rg.ymin-m:rg.ymax+m, rg.xmin-m:rg.xmax+m,...]
                WrongErrs.append(errImg)

    #if debug:
    #if nTP[0] != nPos[0] or nTP[0] != nPred[0] or nTP[1] != nPos[1] or nTP[1] != nPred[1]:
    #if debug or nTP[1] != nPos[1] or nTP[1] != nPred[1]:
    #if True:
    if debug and ((nTP[0] != nPos[0] or nTP[0] != nPred[0]) or (num_classes > 1  and (nTP[1] != nPos[1] or nTP[1] != nPred[1]))):
        print(nPos, nPred, nTP)
        #Masks = []
        #for i in range(yy_seg.shape[-1]):
        #    Masks.append(yy_seg[...,i]*255)
        #ViewImgs(Masks)

        #cv2_imshow(y_img)
        cv2_imshow(yy_img)
        #ViewImgs([yy_img])
        #y_img = MasksImg(all_masks, img)
        #y_det_img = MasksImg(det_masks, img)
        #yy_det_img = MasksImg(yy_det_masks, img)
        #ViewImgs([y_img, yy_img, yy_det_img])
        #ViewImgs([y_det_img, yy_det_img])
        if FileName is not None:
            SaveImgs([yy_img], FileName)

    return nPos, nPred, nTP, MissErrs, WrongErrs


def ModelTest(model, FileNames, num_classes, thresh=0.7, debug=False):
    print("Testing...")
    nPositive = np.zeros(num_classes)
    nPredicted = np.zeros(num_classes)
    nTruePos = np.zeros(num_classes)
    MissErrs, WrongErrs = [], []
    for (idx, img_fn) in enumerate(FileNames):
        nPos, nPred, nTP, Miss, Wrong = SampleTest(model, img_fn, num_classes=num_classes, thresh=thresh, debug=debug)
        if nPos is not None:
            nPositive += nPos
            nPredicted += nPred
            nTruePos += nTP

            # inspect result
            if (nTP[0] != nPos[0] or nTP[0] != nPred[0]) or (num_classes > 1  and (nTP[1] != nPos[1] or nTP[1] != nPred[1])):
            #if nPos[0] != nTP[0]:
                print(idx, nPos, nPred, nTP)
                if len(MissErrs) < 500:
                    MissErrs.extend(Miss)
                if len(WrongErrs) < 500:
                    WrongErrs.extend(Wrong)

    # end of for
    if len(FileNames) > 0:
        Recall = nTruePos/nPositive
        Precision = nTruePos/nPredicted
        F1  = []
        for i in range(len(Precision)):
          F1.append(2*Precision[i]*Recall[i]/(Precision[i] + Recall[i]) if Precision[i] + Recall[i] > 0 else 0)

        print("Class: Recall, Precision, F1")
        for i in range(num_classes):
            #print("%s: %.4f, %.4f, %.4f" % (class_names[i], Recall[i], Precision[i], F1[i]))
            print("%s: %.2f (%.2f, %.2f)" % (class_names[i], F1[i]*100, Precision[i]*100, Recall[i]*100))


    print(nPositive, nPredicted, nTruePos)
    nTP = sum(nTruePos)
    nPr = sum(nPredicted)
    nPos = sum(nPositive)
    P, R = nTP / nPr, nTP / nPos
    F = 2*P*R/(P + R)
    print("All: %.2f (%.2f, %.2f)" % (F*100, P*100, R*100))

    # errors
    SaveErrs(MissErrs, 'MissErrs')
    SaveErrs(WrongErrs, 'WrongErrs')

    #return nPositive, nPredicted, nTruePos, MissErrs, WrongErrs


def GetRegionsPD(masks, nClass, x):
    code = reverse_one_hot(masks)
    x_bw = Binary(x)
    Regions = []
    for c in range(nClass):
        #c_mask = np.uint8((code == c))*255
        c_mask = np.uint8((masks[...,c] >= 0.5))*255

        #c_mask = cv2.dilate(c_mask, np.ones((4, 4), np.uint8))
        #if c == 1: # isolated formula
        #  c_mask = cv2.morphologyEx(c_mask, cv2.MORPH_CLOSE, np.ones((7,31),np.uint8))
        #elif c == 0: # embeded formula
        #  c_mask = cv2.morphologyEx(c_mask, cv2.MORPH_CLOSE, np.ones((3, 7), np.uint8))

        #ViewImgs([c_mask])

        nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(c_mask)
        c_regions = []
        for i in range(1, nlabels):
            l_mask = (labels == i).astype("uint8") * 255
            l, t = stats[i, cv2.CC_STAT_LEFT], stats[i, cv2.CC_STAT_TOP]
            w, h = stats[i, cv2.CC_STAT_WIDTH], stats[i, cv2.CC_STAT_HEIGHT]
            r, b = l + w, t + h

            if (c == 0 and w >= MIN_W and h >= MIN_H) or (c == 1 and w > 32 and h >= 16):
                rg = Region(str(c), l-2, t-2, r+2, b+2)
                if c == 0 and rg.ymax - rg.ymin > 30 and rg.xmax - rg.xmin > 30:
                #if False:
                    rgs = SplitFormula(rg, x_bw, l_mask)
                    if len(rgs) > 1:
                        c_regions.extend(rgs)
                    else:
                        c_regions.append(rg)
                else:
                    c_regions.append(rg)

        #c_mask = cv2.dilate(c_mask, np.ones((3, 3), np.uint8))
        c_regions = AdjustPDs(c_regions, x_bw, c_mask)
        Regions.extend(c_regions)

    #ViewImgs([x_img])
    Regions = PostProcess(Regions, x_bw)

    return Regions

def SplitFormula(reg, x, mask):
    #msk = (mask[reg.ymin:reg.ymax, reg.xmin:reg.xmax]*255).astype(np.uint8)
    msk = mask[reg.ymin:reg.ymax, reg.xmin:reg.xmax]
    img = x[reg.ymin:reg.ymax, reg.xmin:reg.xmax]
    img = 255-(255-img)*(msk==255)

    #ViewImgs([msk, img])

    img2 = cv2.erode(img,np.ones((1,7)))
    img3 = cv2.dilate(img2,np.ones((3,1)))
    hist = np.sum((255-img3)/255,axis=1)

    #plt.figure().set_figheight(2.2)
    #ys = [y for y in range(len(hist))]
    #plt.gca().invert_yaxis()
    #plt.plot(hist, ys)

    #plt.show()

    s = 16
    e = len(hist) - 16
    for i in range(s,e):
      if hist[i] == 0:
        msk[i,] = 0

    #msk = (msk*255).astype(np.uint8)
    new_regions = []
    # candidate CCs
    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(msk)

    # view CCs
    # Map component labels to hue val, 0-179 is the hue range in OpenCV
    label_hue = np.uint8(100*labels/np.max(labels))
    blank_ch = 255*np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

    # Converting cvt to BGR
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2RGB)

    # set bg label to black
    labeled_img[label_hue==0] = 0

    #ViewImgs([img,labeled_img])

    for i in range(1, nlabels):
        l, t = stats[i, cv2.CC_STAT_LEFT], stats[i, cv2.CC_STAT_TOP]
        w, h = stats[i, cv2.CC_STAT_WIDTH], stats[i, cv2.CC_STAT_HEIGHT]
        # check and get sub-regions
        if w >= MIN_W and h >= MIN_H:
          l += reg.xmin
          t += reg.ymin
          r, b = l + w, t + h
          rg = Region(reg.tag, l, t, r, b, None)
          new_regions.append(rg)

    #print(len(new_regions))
    #ViewImgs([img, img2, img3, msk])

    return new_regions


# adjusted PD
def AdjustPDs(regions, bw, mask=None):
    for i, rg in enumerate(regions):
      if int(rg.tag) == 0:
        regions[i] = AdjustPD(rg, bw, mask)
      else:
        regions[i] = AdjustPD(rg, bw)

    regions = RemoveNested(regions)
    return regions


def AdjustPD(rg, bw, mask=None):
    # left-right
    roi = bw[rg.ymin:rg.ymax, rg.xmin:rg.xmax]
    if mask is not None:
        msk = mask[rg.ymin:rg.ymax, rg.xmin:rg.xmax]
        roi = 255-(255-roi)*(msk==255)

    hist = np.sum((255-roi)/255,axis=0)
    len_h = len(hist)

    # find the closest white line
    left, right = 0, len_h-1
    while left < right and hist[left] == 0:
        left += 1
    left -= 1
    while right > left and hist[right] == 0:
        right -= 1
    right += 1
    rg.xmin += left
    rg.xmax -= (len_h - right)

    # top-bottom
    roi = bw[rg.ymin:rg.ymax, rg.xmin:rg.xmax]
    if mask is not None:
        msk = mask[rg.ymin:rg.ymax, rg.xmin:rg.xmax]
        roi = 255-(255-roi)*(msk==255)

    #ViewImgs([roi])
    hist = np.sum((255-roi)/255,axis=1)
    len_h = len(hist)

    # find the closest white line
    top, bot = 0, len_h-1
    while top < bot and hist[top] == 0:
        top += 1
    top -= 1
    while bot > top and hist[bot] == 0:
        bot -= 1
    bot +=1
    rg.ymin += top
    rg.ymax -= (len_h - bot)

    return rg

def AdjustPD0(rg, bw, mask=None):
    roi = bw[rg.ymin:rg.ymax, rg.xmin:rg.xmax]
    if mask is not None:
      msk = mask[rg.ymin:rg.ymax, rg.xmin:rg.xmax]
      roi = 255-(255-roi)*(msk==255)

    #ViewImgs([roi])
    # top and bottom
    hist = np.sum((255-roi)/255,axis=1)
    top, bot = 0, len(hist)-1
    # top
    while top < bot:
      if hist[top] > 0:
        break
      top += 1
    # bottom
    while bot > top:
      if hist[bot] > 0:
        break
      bot -= 1
    rg.ymax = rg.ymin + bot + 1
    rg.ymin = rg.ymin + top

    # left and right
    hist = np.sum((255-roi)/255,axis=0)
    left, right = 0, len(hist)-1
    # top
    while left < right:
      if hist[left] > 0:
        break
      left += 1
    # bottom
    while right > left:
      if hist[right] > 0:
        break
      right -= 1
    rg.xmax = rg.xmin + right + 1
    rg.xmin = rg.xmin + left

    return rg

def GetErrImgs(img, RegionsGT, RegionsPD, num_classes, thresh, debug=False):
    if img.shape[-1] == 1:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

    for rg in RegionsGT:
      cv2.rectangle(img, (rg.xmin, rg.ymin), (rg.xmax, rg.ymax), (0,0,255), 1)
    for rg in RegionsPD:
      cv2.rectangle(img, (rg.xmin, rg.ymin), (rg.xmax, rg.ymax), (255,0,0), 1)

    # errors
    m = 8
    MissErrs, WrongErrs = [], []
    # miss errors
    IoUs, TPRegs, nPos, nPred, nTP = MatchRegs(RegionsGT, RegionsPD, num_classes, thresh)
    for iou, rg in zip(IoUs, RegionsGT):
      if iou < thresh:
        cv2.putText(img, "{:.1f}".format(iou*100), (rg.xmin, rg.ymin-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)

        # err images
        if int(rg.tag) == 0:
            errImg = img[rg.ymin-m:rg.ymax+m, rg.xmin-m:rg.xmax+m,...]
            MissErrs.append(errImg)

    # wrong errors
    IoUs, TPRegs, nPos, nPred, nTP = MatchRegs(RegionsPD, RegionsGT, num_classes, thresh)
    for iou, rg in zip(IoUs, RegionsPD):
      if iou < thresh:
        cv2.putText(img, "{:.1f}".format(iou*100), (rg.xmin, rg.ymin-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1)

        # err images
        errImg = img[rg.ymin-m:rg.ymax+m, rg.xmin-m:rg.xmax+m,...]
        if int(rg.tag) == 0:
            errImg = img[rg.ymin-m:rg.ymax+m, rg.xmin-m:rg.xmax+m,...]
            WrongErrs.append(errImg)

    Imgs = [img]

    return Imgs, MissErrs, WrongErrs


# match regions
def MatchRegs(GTRegs, PDRegs, nClass, thresh):
    IoUs = []
    TPRegs = []
    for reg in GTRegs:
      MaxIoU = Match(reg, PDRegs)
      IoUs.append(MaxIoU)

      if MaxIoU >= thresh:
        TPRegs.append(reg)

    nPos = np.zeros(nClass)
    nPD = np.zeros(nClass)
    nTP = np.zeros(nClass)
    for i in range(nClass):
      nPos[i] = len([r for r in GTRegs if int(r.tag) == i])
      nPD[i]  = len([r for r in PDRegs if int(r.tag) == i])
      nTP[i]  = len([r for r in TPRegs if int(r.tag) == i])

    return IoUs, TPRegs, nPos, nPD, nTP

def Match(rg, Regs):
    MaxIoU = 0.0
    for reg in Regs:
      if rg.tag == reg.tag:
        iou = bb_iou(rg, reg)
        if MaxIoU < iou:
          MaxIoU = iou

    return MaxIoU

def PostProcess(Regions, bw):
    # remove small and nested
    Regions = RemoveSmall(Regions)
    Regions = RemoveNested(Regions)

    # group isolated
    Regions = GroupIsolated(Regions, bw)

    # combine embbed isolated
    Regions = MergeEI(Regions, bw)

    return Regions


def RemoveSmall(Regions):
    NewRegions = []
    for rg in Regions:
        if not IsSmall(rg):
          NewRegions.append(rg)

    return NewRegions

MIN_EMB_WIDTH, MIN_EMB_HEIGHT = 6, 8
MIN_ISO_WIDTH, MIN_ISO_HEIGHT = 32, 16
def IsSmall(rg):
    if (int(rg.tag) == 0 and (rg.xmax - rg.xmin < MIN_EMB_WIDTH or rg.ymax - rg.ymin < MIN_EMB_HEIGHT)) or \
       (int(rg.tag) == 1 and (rg.xmax - rg.xmin < MIN_ISO_WIDTH or rg.ymax - rg.ymin < MIN_ISO_HEIGHT)):
       return True

    return False


def RemoveNested(Regions):
    NewRegions = []
    for i in range(len(Regions)):
        if not IsNested(i, Regions):
          NewRegions.append(Regions[i])

    return NewRegions

def IsNested(i, regions):
    ri = regions[i]
    for j in range(len(regions)):
      if j != i:
        rj = regions[j]
        if rj.xmin <= ri.xmin+2 and ri.xmax <= rj.xmax+2 and rj.ymin <= ri.ymin+2 and ri.ymax <= rj.ymax+2:
          return True

    return False

def IsIsoMergeable(rg1, rg2, bw=None):
    return IsHorOverlapB(rg1, rg2) and IsVerOverlap(rg1, rg2, 96)

def GroupIsolated(Regions, bw):
    NewRegions = []
    nReg = len(Regions)
    for (i,rg) in enumerate(Regions):
        if rg.tag == '1':
            for j in range(i+1, nReg):
                if Regions[j].tag == '1' and IsIsoMergeable(rg, Regions[j], bw):
                    rg = Merge(rg, Regions[j])
                    Regions[j].tag = '-1'
                    break
            NewRegions.append(rg)
        elif rg.tag == '0':
            NewRegions.append(rg)

    return NewRegions

def IsEIMergeable(rg1, rg2, bw=None):
    return IsHorOverlapC(rg1, rg2) and IsVerOverlap(rg1, rg2)

def MergeEI(Regions, bw):
    NewRegions = []
    nReg = len(Regions)
    for (i,rg) in enumerate(Regions):
        if rg.tag == '0':
            for j in range(i+1, nReg):
                if Regions[j].tag == '1' and IsEIMergeable(rg, Regions[j], bw):
                    if Regions[j].xmax - Regions[j].xmin > rg.xmax - rg.xmin:
                        rg.tag = Regions[j].tag
                    rg = Merge(rg, Regions[j])
                    Regions[j].tag = '-1'
                    break
            NewRegions.append(rg)
        elif rg.tag == '1':
            NewRegions.append(rg)

    return NewRegions


def Merge(rg1, rg2):
    rg1.xmin = min(rg1.xmin, rg2.xmin)
    rg1.ymin = min(rg1.ymin, rg2.ymin)
    rg1.xmax = max(rg1.xmax, rg2.xmax)
    rg1.ymax = max(rg1.ymax, rg2.ymax)

    return rg1

def IsHorOverlapC(rg1, rg2, d=0):
    c1 = (rg1.ymin + rg1.ymax) // 2
    c2 = (rg2.ymin + rg2.ymax) // 2
    return (rg1.ymin-d <= c2 and c2 <= rg1.ymax+d) and \
           (rg2.ymin-d <= c1 and c1 <= rg2.ymax+d)


def IsHorOverlapB(rg1, rg2, d=0):
    return (rg1.ymin-d <= rg2.ymin and rg2.ymin <= rg1.ymax+d) or \
           (rg1.ymin-d <= rg2.ymax and rg2.ymax <= rg1.ymax+d) or \
           (rg2.ymin-d <= rg1.ymin and rg1.ymin <= rg2.ymax+d) or \
           (rg2.ymin-d <= rg1.ymax and rg1.ymax <= rg2.ymax+d)

def IsVerOverlap(rg1, rg2, d=0):
    return (rg1.xmin-d <= rg2.xmin and rg2.xmin <= rg1.xmax+d) or \
           (rg1.xmin-d <= rg2.xmax and rg2.xmax <= rg1.xmax+d) or \
           (rg2.xmin-d <= rg1.xmin and rg1.xmin <= rg2.xmax+d) or \
           (rg2.xmin-d <= rg1.xmax and rg1.xmax <= rg2.xmax+d)


# IBEM data...

In [ ]:
# copy and decompress data
!mkdir IBEM

# train
!mkdir IBEM/Tr00
!mkdir IBEM/Tr00/images
!mkdir IBEM/Tr01
!mkdir IBEM/Tr01/images
!mkdir IBEM/Tr10
!mkdir IBEM/Tr10/images

!unzip drive/MyDrive/Data/IBEM/Tr00/Tr00.zip -d IBEM/Tr00/images > /dev/null
!unzip drive/MyDrive/Data/IBEM/Tr00/labels.zip -d IBEM/Tr00 > /dev/null

!unzip drive/MyDrive/Data/IBEM/Tr01/Tr01.zip -d IBEM/Tr01/images > /dev/null
!unzip drive/MyDrive/Data/IBEM/Tr01/labels.zip -d IBEM/Tr01 > /dev/null

!unzip drive/MyDrive/Data/IBEM/Tr10/Tr10.zip -d IBEM/Tr10/images > /dev/null
!unzip drive/MyDrive/Data/IBEM/Tr10/labels.zip -d IBEM/Tr10 > /dev/null

# validation
!mkdir IBEM/Va00
!mkdir IBEM/Va00/images
!mkdir IBEM/Va01
!mkdir IBEM/Va01/images

!unzip drive/MyDrive/Data/IBEM/Va00/Va00.zip -d IBEM/Va00/images > /dev/null
!unzip drive/MyDrive/Data/IBEM/Va00/labels.zip -d IBEM/Va00 > /dev/null

!unzip drive/MyDrive/Data/IBEM/Va01/Va01.zip -d IBEM/Va01/images > /dev/null
!unzip drive/MyDrive/Data/IBEM/Va01/labels.zip -d IBEM/Va01 > /dev/null

# test
!mkdir IBEM/Ts00
!mkdir IBEM/Ts00/images
!mkdir IBEM/Ts01
!mkdir IBEM/Ts01/images
!mkdir IBEM/Ts10
!mkdir IBEM/Ts10/images
!mkdir IBEM/Ts11
!mkdir IBEM/Ts11/images

!unzip drive/MyDrive/Data/IBEM/Ts00/Ts00.zip -d IBEM/Ts00/images > /dev/null
!unzip drive/MyDrive/Data/IBEM/Ts00/labels.zip -d IBEM/Ts00 > /dev/null

!unzip drive/MyDrive/Data/IBEM/Ts01/Ts01.zip -d IBEM/Ts01/images > /dev/null
!unzip drive/MyDrive/Data/IBEM/Ts01/labels.zip -d IBEM/Ts01 > /dev/null

!unzip drive/MyDrive/Data/IBEM/Ts10/Ts10.zip -d IBEM/Ts10/images > /dev/null
!unzip drive/MyDrive/Data/IBEM/Ts10/labels.zip -d IBEM/Ts10 > /dev/null

!unzip drive/MyDrive/Data/IBEM/Ts11/Ts11.zip -d IBEM/Ts11/images > /dev/null
!unzip drive/MyDrive/Data/IBEM/Ts11/labels.zip -d IBEM/Ts11 > /dev/null

In [ ]:
Folder_Tr00 = '/content/IBEM/Tr00/images'
Folder_Tr01 = '/content/IBEM/Tr01/images'
Folder_Tr10 = '/content/IBEM/Tr10/images'

Folder_Va00 = '/content/IBEM/Va00/images'
Folder_Va01 = '/content/IBEM/Va01/images'

Folder_Ts00 = '/content/IBEM/Ts00/images'
Folder_Ts01 = '/content/IBEM/Ts01/images'
Folder_Ts10 = '/content/IBEM/Ts10/images'
Folder_Ts11 = '/content/IBEM/Ts11/images'

AllFiles = GetFiles([Folder_Tr00, Folder_Tr01, Folder_Tr10, Folder_Va00, Folder_Va01, Folder_Ts00, Folder_Ts01, Folder_Ts10, Folder_Ts11], 'jpg')
print(len(AllFiles))

Tr00Files = GetFiles([Folder_Tr00], 'jpg')
Ts10Files = GetFiles([Folder_Ts10], 'jpg')
Ts11Files = GetFiles([Folder_Ts11], 'jpg')


In [ ]:
import yaml

def GetLabel(yaml_file):
    class_names = []
    with open(yaml_file, "r", encoding="utf-8") as stream:
        try:
            data_loaded = yaml.safe_load(stream)
            #print(data_loaded['names'])
            class_names = list(data_loaded['names'].values())
        except yaml.YAMLError as exc:
            print(exc)
    return class_names


def Txt2Regions(txt_fn, img):
    img_h, img_w = img.shape[:2]
    # ground truth math expressions
    Regions = []
    with open(txt_fn, 'r') as f:
        for i, line in enumerate(f.readlines()):
            line = line.strip('\n').split(' ')
            label = int(float(line[0]))

            nPoints = (len(line)-1)//2
            if nPoints == 2: # detection
                x, y = float(line[1]), float(line[2])
                w, h = float(line[3]), float(line[4])
                l, t = x - w/2, y - h/2
                r, b = x + w/2, y + h/2
                l, t = int(l*img_w), int(t*img_h)
                r, b = int(r*img_w), int(b*img_h)

                Regions.append(Region(str(label), l, t, r, b))
            else: # segmentation
                cnt, cnt2 = [], []
                for p in range(nPoints):
                    x = float(line[2*p+1])
                    y = float(line[2*p+2])
                    cnt.append(int(x*img_w+1))
                    cnt.append(int(y*img_h-1))
                    cnt2.append([int(x*img_w+1), int(y*img_h-1)])

                cnt = np.asarray(cnt)
                cnt2 = np.asarray(cnt2)
                x,y,w,h = cv2.boundingRect(cnt2)
                Regions.append(Region(str(label), x, y, x+w, y+h, poly=cnt))

    return Regions

data_yaml = '/content/drive/MyDrive/Data/IBEM/IBEM.yaml'
class_names = GetLabel(data_yaml)
num_classes = len(class_names)
print(class_names)

def IBEMLoad(img_fn):
    # image
    img = cv2.imread(img_fn)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # regions
    txt_fn = img_fn.replace('images', 'labels').replace('.jpg', '.txt')
    RegionsGT = Txt2Regions(txt_fn, img)

    # adjust regions
    bw = Binary(img)
    RegionsGT = AdjustGTs(RegionsGT, bw)

    return img, RegionsGT

# View data
ViewData(Ts11Files[115], IBEMLoad)

# ...and test